<i>Copyright (c) Microsoft Corporation. All rights reserved.</i>

<i>Licensed under the MIT License.</i>

#  Mind Utils Generation

Many news recommendation methods ultilize word embeddings, news vertical embeddings, news subvertical embeddings and user id embedding. Therefore, it is necessary to generate a word dictionary, a vertical dictionary, a subvertical dictionary and a userid dictionary to convert words, news verticals, subvericals and user ids from strings to indexes. To use the pretrain word embedding, a embedding matrix is generated as the intial weight of the word embedding layer.


This notebook gives examples about how to generate
* word_dict.pkl: convert the words in news titles into indexes.
* word_dict_all.pkl: convert the words in news titles and abstracts into indexes.
* embedding.npy: pretrained word embedding matrix of words in word_dict.pkl
* embedding_all.npy: pretrained embedding matrix of words in word_dict_all.pkl
* vert_dict.pkl: convert news verticals into indexes.
* subvert_dict.pkl: convert news subverticals into indexes.
* uid2index.pkl: convert user ids into indexes.

In [1]:
import sys
import os
import pandas as pd
from collections import Counter
from tqdm import tqdm
import pickle
import numpy as np
import scrapbook as sb

from tempfile import TemporaryDirectory
from recommenders.datasets.mind import (download_mind,
                                     extract_mind,
                                     download_and_extract_glove,
                                     load_glove_matrix,
                                     word_tokenize
                                    )
from recommenders.datasets.download_utils import unzip_file

print("System version: {}".format(sys.version))


System version: 3.8.5 (default, Sep  4 2020, 07:30:14) 
[GCC 7.3.0]


In [2]:
mind_type="demo"
# word_embedding_dim should be in [50, 100, 200, 300]
word_embedding_dim = 300

In [3]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name
train_zip, valid_zip = download_mind(size=mind_type, dest_path=data_path)
unzip_file(train_zip, os.path.join(data_path, 'train'), clean_zip_file=False)
unzip_file(valid_zip, os.path.join(data_path, 'valid'), clean_zip_file=False)
output_path = os.path.join(data_path, 'utils')
os.makedirs(output_path, exist_ok=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17.0k/17.0k [00:00<00:00, 63.0kKB/s]


## Prepare utils of news

* word dictionary
* vertical dictionary
* subvetical dictionary

In [4]:
news = pd.read_table(os.path.join(data_path, 'train', 'news.tsv'),
                     names=['newid', 'vertical', 'subvertical', 'title',
                            'abstract', 'url', 'entities in title', 'entities in abstract'],
                     usecols = ['vertical', 'subvertical', 'title', 'abstract'])

In [5]:
news.head()

,vertical,subvertical,title,abstract
0,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the..."
1,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...
2,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi..."
3,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re..."
4,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...


In [6]:
news_vertical = news.vertical.drop_duplicates().reset_index(drop=True)
vert_dict_inv = news_vertical.to_dict()
vert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

news_subvertical = news.subvertical.drop_duplicates().reset_index(drop=True)
subvert_dict_inv = news_subvertical.to_dict()
subvert_dict = {v: k+1 for k, v in vert_dict_inv.items()}

In [7]:
news.title = news.title.apply(word_tokenize)
news.abstract = news.abstract.apply(word_tokenize)

In [8]:
word_cnt = Counter()
word_cnt_all = Counter()

for i in tqdm(range(len(news))):
    word_cnt.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['title'])
    word_cnt_all.update(news.loc[i]['abstract'])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26740/26740 [00:03<00:00, 7646.21it/s]


In [9]:
word_dict = {k: v+1 for k, v in zip(word_cnt, range(len(word_cnt)))}
word_dict_all = {k: v+1 for k, v in zip(word_cnt_all, range(len(word_cnt_all)))}

In [10]:
with open(os.path.join(output_path, 'vert_dict.pkl'), 'wb') as f:
    pickle.dump(vert_dict, f)
    
with open(os.path.join(output_path, 'subvert_dict.pkl'), 'wb') as f:
    pickle.dump(subvert_dict, f)

with open(os.path.join(output_path, 'word_dict.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)
    
with open(os.path.join(output_path, 'word_dict_all.pkl'), 'wb') as f:
    pickle.dump(word_dict, f)

## Prepare embedding matrixs
* embedding.npy
* embedding_all.npy

In [11]:
glove_path = download_and_extract_glove(data_path)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 842k/842k [02:55<00:00, 4.79kKB/s]


In [12]:
embedding_matrix, exist_word = load_glove_matrix(glove_path, word_dict, word_embedding_dim)
embedding_all_matrix, exist_all_word = load_glove_matrix(glove_path, word_dict_all, word_embedding_dim)

400000it [00:08, 47655.60it/s]
400000it [00:09, 42770.60it/s]


In [21]:
embedding_all_matrix

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.04656  ,  0.21318  , -0.0074364, ...,  0.0090611, -0.20989  ,
         0.053913 ],
       [ 0.42362  ,  0.9854   ,  0.27036  , ...,  0.55034  , -0.25911  ,
        -0.21395  ],
       ...,
       [ 0.4606   , -0.10309  ,  0.71421  , ...,  0.52476  , -0.27522  ,
        -0.046025 ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [-0.11625  , -0.22006  ,  0.63804  , ...,  0.32741  , -0.15666  ,
         0.058902 ]])

In [22]:
exist_all_word

['the',
 ',',
 '.',
 'of',
 'to',
 'and',
 'in',
 'a',
 'for',
 'that',
 'on',
 'is',
 'was',
 'said',
 'with',
 'he',
 'as',
 'it',
 'by',
 'at',
 'from',
 'his',
 'an',
 'be',
 'has',
 'are',
 'have',
 'but',
 'were',
 'not',
 'this',
 'who',
 'they',
 'had',
 'i',
 'which',
 'will',
 'their',
 'or',
 'its',
 'one',
 'after',
 'new',
 'been',
 'also',
 'we',
 'would',
 'two',
 'more',
 'first',
 'about',
 'up',
 'when',
 'year',
 'there',
 'all',
 'out',
 'she',
 'other',
 'people',
 'her',
 'percent',
 'than',
 'over',
 'into',
 'last',
 'some',
 'government',
 'time',
 'you',
 'years',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 ';',
 'president',
 'only',
 'state',
 'million',
 'could',
 'us',
 'most',
 '_',
 'against',
 'so',
 'them',
 'what',
 'him',
 'united',
 'during',
 'before',
 'may',
 'since',
 'many',
 'while',
 'where',
 'states',
 'because',
 'now',
 'city',
 'made',
 'like',
 'between',
 'did',
 'just',
 'national',
 'day',
 'country',
 'under',
 'such',
 'secon

In [13]:
np.save(os.path.join(output_path, 'embedding.npy'), embedding_matrix)
np.save(os.path.join(output_path, 'embedding_all.npy'), embedding_all_matrix)

## Prepare uid2index.pkl

In [14]:
uid2index = {}

with open(os.path.join(data_path, 'train', 'behaviors.tsv'), 'r') as f:
    for l in tqdm(f):
        uid = l.strip('\n').split('\t')[1]
        if uid not in uid2index:
            uid2index[uid] = len(uid2index) + 1

22034it [00:00, 485752.32it/s]


In [19]:
uid2index

{'U82271': 1,
 'U84185': 2,
 'U11552': 3,
 'U68381': 4,
 'U52303': 5,
 'U26536': 6,
 'U47186': 7,
 'U63195': 8,
 'U69242': 9,
 'U70695': 10,
 'U86374': 11,
 'U72944': 12,
 'U13132': 13,
 'U90734': 14,
 'U57329': 15,
 'U75773': 16,
 'U84983': 17,
 'U22478': 18,
 'U46200': 19,
 'U87270': 20,
 'U33585': 21,
 'U57826': 22,
 'U93213': 23,
 'U51767': 24,
 'U48080': 25,
 'U75917': 26,
 'U26276': 27,
 'U22697': 28,
 'U87145': 29,
 'U88771': 30,
 'U88960': 31,
 'U35178': 32,
 'U60646': 33,
 'U66821': 34,
 'U58380': 35,
 'U89410': 36,
 'U92961': 37,
 'U40469': 38,
 'U30198': 39,
 'U92846': 40,
 'U70985': 41,
 'U82568': 42,
 'U20030': 43,
 'U82748': 44,
 'U4539': 45,
 'U51779': 46,
 'U53425': 47,
 'U62843': 48,
 'U39322': 49,
 'U49116': 50,
 'U60991': 51,
 'U6340': 52,
 'U60681': 53,
 'U49608': 54,
 'U46642': 55,
 'U44210': 56,
 'U22199': 57,
 'U4501': 58,
 'U67562': 59,
 'U24392': 60,
 'U91923': 61,
 'U1150': 62,
 'U83940': 63,
 'U82485': 64,
 'U14462': 65,
 'U25647': 66,
 'U9603': 67,
 'U6519':

In [15]:
with open(os.path.join(output_path, 'uid2index.pkl'), 'wb') as f:
    pickle.dump(uid2index, f)

In [16]:
utils_state = {
    'vert_num': len(vert_dict),
    'subvert_num': len(subvert_dict),
    'word_num': len(word_dict),
    'word_num_all': len(word_dict_all),
    'embedding_exist_num': len(exist_word),
    'embedding_exist_num_all': len(exist_all_word),
    'uid2index': len(uid2index)
}

In [20]:
utils_state

{'vert_num': 17,
 'subvert_num': 17,
 'word_num': 23404,
 'word_num_all': 41074,
 'embedding_exist_num': 22408,
 'embedding_exist_num_all': 37634,
 'uid2index': 5000}

In [17]:
sb.glue("utils_state", utils_state)

In [18]:
tmpdir.cleanup()